In [1]:
from sklearn.model_selection import TimeSeriesSplit

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("datasetslagfortraining.csv")

In [5]:
df.info

<bound method DataFrame.info of       Unnamed: 0             geography        stratum        date  \
0             12         East Midlands  Glycopeptides  2021-08-01   
1             13         East Midlands  Glycopeptides  2021-09-01   
2             14         East Midlands  Glycopeptides  2021-10-01   
3             15         East Midlands  Glycopeptides  2021-11-01   
4             16         East Midlands  Glycopeptides  2021-12-01   
...          ...                   ...            ...         ...   
9589       11317  Yorkshire and Humber     Penicillin  2025-02-01   
9590       11318  Yorkshire and Humber     Penicillin  2025-03-01   
9591       11319  Yorkshire and Humber     Penicillin  2025-04-01   
9592       11320  Yorkshire and Humber     Penicillin  2025-05-01   
9593       11321  Yorkshire and Humber     Penicillin  2025-06-01   

      resistance_pct      organism  \
0              43.67    e_faecalis   
1              42.67    e_faecalis   
2              38.33    e

In [6]:
window_length = 12

In [7]:
feature_cols = ([f'lag_{i}' for i in range (1, window_length +1)] + [f'presc_lag_{i}'for i in [1,3, 6]] + ['month'] + ['trend'] + ['prescribing_x_month'])

In [9]:
def rolling_backtest (model, df, feature_cols, horizon=6, step=1):
    preds = []
    actuals = []
    dates = []

    all_dates = sorted(df["date"].unique())

    for i in range(24, len(all_dates) - horizon, step):
        train_end = all_dates[i]
        test_start = all_dates[i + 1]
        test_end = all_dates[i + horizon]

        train = df[df["date"] <= train_end]
        test = df[(df["date"] > train_end) & (df["date"] <= test_end)]

        X_train = train[feature_cols]
        y_train = train["resistance_pct"]
        X_test = test[feature_cols]
        y_test = test["resistance_pct"]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        preds.append(y_pred)
        actuals.append(y_test.values)
        dates.append(test["date"].values)
    return np.concatenate(preds), np.concatenate(actuals), np.concatenate(dates)
